In [1]:
pip install requests pandas

In [6]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd

In [3]:
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'
REDIRECT_URI = 'your_redirect_uri'
AUTH_URL = 'https://www.linkedin.com/oauth/v2/authorization'
TOKEN_URL = 'https://www.linkedin.com/oauth/v2/accessToken'

### Get Authorization Code

In [5]:
auth_params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'redirect_uri': REDIRECT_URI,
    'scope': 'r_liteprofile r_emailaddress w_member_social',
}
response = requests.get(AUTH_URL, params=auth_params)
print(f"Visit this URL to authorize the application: {response.url}")

Visit this URL to authorize the application: https://www.linkedin.com/oauth/v2/authorization?response_type=code&client_id=your_client_id&redirect_uri=your_redirect_uri&scope=r_liteprofile+r_emailaddress+w_member_social


### Get Access Token

In [ ]:
auth_code = input('Enter the authorization code: ')
token_params = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': REDIRECT_URI,
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
}

token_response = requests.post(TOKEN_URL, data=token_params, auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET))
access_token = token_response.json().get('access_token')
print(f"Access Token: {access_token}")

In [ ]:
def fetch_job_descriptions(access_token, job_title, location=''):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Restli-Protocol-Version': '2.0.0'
    }

    search_url = 'https://api.linkedin.com/v2/jobSearch'
    params = {
        'keywords': job_title,
        'location': location,
        'count': 10  # Number of job postings to fetch
    }

    response = requests.get(search_url, headers=headers, params=params)
    jobs = response.json().get('elements', [])

    job_descriptions = []
    for job in jobs:
        job_descriptions.append({
            'Title': job.get('title'),
            'Company': job.get('companyName'),
            'Location': job.get('formattedLocation'),
            'Description': job.get('description')
        })

    return pd.DataFrame(job_descriptions)

In [7]:
data_analyst_jobs = fetch_job_descriptions(access_token, 'Data Analyst')
data_engineer_jobs = fetch_job_descriptions(access_token, 'Data Engineer')
data_scientist_jobs = fetch_job_descriptions(access_token, 'Data Scientist')

NameError: name 'fetch_job_descriptions' is not defined

In [ ]:
# Save to CSV
data_analyst_jobs.to_csv('data_analyst_jobs.csv', index=False)
data_engineer_jobs.to_csv('data_engineer_jobs.csv', index=False)
data_scientist_jobs.to_csv('data_scientist_jobs.csv', index=False)